### Ancestral recombination graphs for COVID-19 ORF8



In [21]:
from Bio import SeqIO

# The original fasta is has the unique accession numbers and the descriptions
# backwards. They must be swapped or lastall will have ambiguous output.

with open( 'data/all.fasta' ) as f :
    with open( 'data/all_fixednames.fasta', 'w' ) as out :
        for record in SeqIO.parse( f, format='fasta' ) :
            d = record.id
            record.id = record.description.split('|')[1].strip()
            record.description = d
            out.write( record.format( 'fasta' ) )

In [36]:
!lastdb data/orf8ref -cR01 -p data/QHD43422_orf8.fasta
!lastal data/orf8ref data/all_fixednames.fasta -D100000 -pPAM30 -P 8 -F15 -f BlastTab+ > data/ref_vs_all.lout

In [57]:
!makeblastdb -in data/QHD43422_orf8.fasta -dbtype prot
!blastx -db data/QHD43422_orf8.fasta -query data/all_fixednames.fasta -outfmt 6 > data/ref_vs_all.out



Building a new DB, current time: 07/31/2020 15:40:56
New DB name:   /home/russell/Projects/CoronaARG/data/QHD43422_orf8.fasta
New DB title:  data/QHD43422_orf8.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/russell/Projects/CoronaARG/data/QHD43422_orf8.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.000304937 seconds.


In [43]:
import pandas

#lastal_cols = [ 'query_id', 'subject_id', 'identity', 'alignment_length',
#                'mismatches', 'gap_opens', 'q_start', 'q_end', 's_start',
#                's_end', 'evalue', 'bitscore', 'query_length', 'subject_length',
#                'raw_score' ]

cols = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 
        'qstart', 'qend','sstart', 'send', 'evalue', 'bitscore', ]

df = pandas.read_csv( 'data/ref_vs_all.out', 
                      sep='\t', names=cols, index_col=False )

df.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MN908947,QHD43422.1,100.000,121,0,0,27894,28256,1,121,7.740000e-82,251.0
1,MN908947,QHD43422.1,26.087,23,17,0,6710,6778,9,31,1.300000e+00,19.2
2,MN908947,QHD43422.1,33.333,24,12,1,29482,29553,10,29,2.000000e+00,18.9
3,MN908947,QHD43422.1,29.412,17,12,0,14780,14830,4,20,8.800000e+00,16.9
4,MN938384,QHD43422.1,99.174,121,1,0,27862,28224,1,121,5.520000e-81,249.0


In [67]:
from Bio import SeqIO

with open( 'data/all_fixednames.fasta' ) as f :
    with open( 'data/orf8.fasta', 'w' ) as out :
        for record in SeqIO.parse( f, format='fasta' ) :
            row = df[ (df.qseqid == record.id) & (df.pident > 90) & (df.length > 100) ]
            if len( row ) != 1 : continue
            start, stop = int(row.qstart), int(row.qend )
            if start > stop : continue # skipping negative strand hits
            out.write( record[ start : stop ].format( 'fasta' ) )
            

In [66]:
df[ (df.qseqid == 'CNA0013710') & (df.pident > 90) & (df.length > 100) ]

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
452,CNA0013710,QHD43422.1,99.174,121,1,0,2022,1660,1,121,5.530000e-81,249.0


#### obsolete code below

In [111]:
from Bio import Entrez, SeqIO, Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_dna

def GetORF( accession, orf ) :
    Entrez.email = 'ryneches@lbl.gov'
    
    handle=Entrez.esearch(db="nucleotide", term=accession, retmode="xml")
    result = Entrez.read(handle)

    handle=Entrez.efetch(db="nucleotide", id=result['IdList'][0], retmode="xml")
    genome = Entrez.read(handle)
    
    for feature in genome[0]['GBSeq_feature-table'] :
        if not feature['GBFeature_key'] == 'CDS' : continue
        #print( feature['GBFeature_quals'][0]['GBQualifier_value'] )
        if orf.lower() == feature['GBFeature_quals'][0]['GBQualifier_value'].lower() :
            break
    
    start, stop = [ int(s) for s in feature['GBFeature_location'].strip().split('..') ]
    result = SeqRecord( Seq.Seq( genome[0]['GBSeq_sequence'][ start : stop ], generic_dna ) )
    result.id = accession
    result.description = orf
    return result

In [112]:
import pyprind

progbar = pyprind.ProgBar( open( 'data/all.fasta' ).read().count('>'), 'Fetching ORFs...' )

orfs = []

with open( 'data/all.fasta' ) as f :
    for record in SeqIO.parse( f, format='fasta' ) :
        progbar.update
        orf = GetORF( record.description.split('|')[1].strip(), 'orf8' )
        print( orf )
        orfs.append( orf )

ValueError: invalid literal for int() with base 10: '<1'

In [115]:
Entrez.email = 'ryneches@lbl.gov'
    
handle=Entrez.esearch(db="nucleotide", term='MN938387', retmode="xml")
result = Entrez.read(handle)

handle=Entrez.efetch(db="nucleotide", id=result['IdList'][0], retmode="xml")
genome = Entrez.read(handle)
    
for feature in genome[0]['GBSeq_feature-table'] :
    if not feature['GBFeature_key'] == 'CDS' : continue
    #print( feature['GBFeature_quals'][0]['GBQualifier_value'] )
    if orf.lower() == feature['GBFeature_quals'][0]['GBQualifier_value'].lower() :
        break
    
start, stop = [ int(s) for s in feature['GBFeature_location'].strip().split('..') ]
result = SeqRecord( Seq.Seq( genome[0]['GBSeq_sequence'][ start : stop ], generic_dna ) )
result.id = accession
result.description = orf

NotImplementedError: SeqRecord comparison is deliberately not implemented. Explicitly compare the attributes of interest.

In [145]:
aln.alignment.score

365.0

In [2]:
from Bio import Align
from Bio import SeqIO

aligner = Align.PairwiseAligner()

with open( 'data/MN908947_orf8.fasta' ) as f :
    for orf8 in SeqIO.parse( f, format='fasta' ) :
        pass

with open( 'data/all.fasta' ) as f :
    for record in SeqIO.parse( f, format='fasta' ) :
        aln = aligner.align( orf8.seq, record.seq )
        print( record.id )
             
        for a in sorted( aln ) :
            print("Score = %.1f:" % a.score)
            print(a)
        
        break

Wuhan-Hu-1


OverflowError: number of optimal alignments is larger than 9223372036854775807

In [109]:
with open( 'data/orf8.fasta', 'w' ) as f :
    for record in orfs :
        f.write( record.format( 'fasta' ) )

In [110]:
!clustalo -i data/orf8.fasta -o data/orf8_aln.fasta -v

Using 8 threads
Read 582 sequences (type: DNA) from data/orf8.fasta
Using 84 seeds (chosen with constant stride from length sorted seqs) for mBed (from a total of 582 sequences)
Calculating pairwise ktuple-distances...
Ktuple-distance calculation progress done. CPU time: 7.64u 0.01s 00:00:07.65 Elapsed: 00:00:04
mBed created 1 cluster/s (with a minimum of 1 and a soft maximum of 100 sequences each)
Distance calculation within sub-clusters done. CPU time: 28.16u 0.04s 00:00:28.20 Elapsed: 00:00:13
Guide-tree computation (mBed) done.
Progressive alignment progress done. CPU time: 13.37u 0.62s 00:00:13.98 Elapsed: 00:00:14
Alignment written to data/orf8_aln.fasta
